# Praca domowa 3

## Ładowanie podstawowych pakietów

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold # used in crossvalidation
from sklearn.model_selection import KFold

import IPython
from time import time

## Krótki wstęp

Celem zadania jest by bazujac na danych meteorologicznych z australi sprawdzić i wytrenowac 3 różne modele. Równie ważnym celem zadania jest przejrzenie oraz zmiana tzn. hiperparamterów z każdego nich. 

### Załadowanie danych

In [ ]:
data = pd.read_csv("../../australia.csv")

### Przyjrzenie się danym

In [ ]:
data.info()

Nie ma w danych żadnych braków, oraz są one przygotowane idealnie do uczenia maszynowego. Przyjżyjmy się jednak jak wygląda ramka. 

In [ ]:
data.head()

## Random Forest

**Załadowanie potrzebnych bibliotek** 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

**Inicjalizowanie modelu**

In [ ]:
rf_default  = RandomForestClassifier()

**Hiperparametry**

In [ ]:
params = rf_default.get_params()
params

**Zmiana kilku hiperparametrów**

In [ ]:
params['n_estimators']=150
params['max_depth']=6
params['min_samples_leaf']=4
params['n_jobs']=4
params['random_state']=0

In [ ]:
rf_modified = RandomForestClassifier()
rf_modified.set_params(**params)

## Extreme Gradient Boosting

**Załadowanie potrzebnych bibliotek** 

In [ ]:
from xgboost import XGBClassifier

**Inicjalizowanie modelu**

In [ ]:
xgb_default = XGBClassifier()

**Hiperparametry**

In [ ]:
params = xgb_default.get_params()
params

**Zmiana kilku hiperparametrów**

In [ ]:
params['n_estimators']=150
params['max_depth']=6
params['n_jobs']=4
params['random_state']=0

In [ ]:
xgb_modified = XGBClassifier()
xgb_modified.set_params(**params)

## Support Vector Machines

**Załadowanie potrzebnych bibliotek** 

In [ ]:
from sklearn.svm import SVC

**Inicjalizowanie modelu**

In [ ]:
svc_default = SVC()

**Hiperparametry**

In [ ]:
params = svc_default.get_params()
params

**Zmiana kilku hiperparametrów**

In [ ]:
params['degree']=3
params['tol']=0.001
params['random_state']=0

In [ ]:
svc_modified = SVC()
svc_modified.set_params(**params)

## Komentarz
W tym momencie otrzymaliśmy 3 modele z zmienionymi hiperparametrami, oraz ich domyślne odpowiedniki. Zobaczmy teraz jak zmieniły się rezultaty osiągane przez te modele i chociaż nie był to cel tego zadania, zobaczmy czy może udało nam się poprawić jakiś model.

## Porównanie

**Załadowanie potrzebnych bibliotek** 

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

### Funkcje pomocnicze

In [ ]:
def cv_classifier(classifier,kfolds = 10, X = data.drop("RainTomorrow", axis = 1), y = data.RainTomorrow):
    start_time = time()
    
    scores ={}
    scores["f1"]=[]
    scores["accuracy"]=[]
    scores["balanced_accuracy"]=[]
    scores["precision"]=[]
    scores["average_precision"]=[]
    scores["roc_auc"]=[]
    
    # Hardcoded crossvalidation metod, could be 
    cv= StratifiedKFold(n_splits=kfolds,shuffle=True,random_state=0)
    
    for i, (train, test) in enumerate(cv.split(X, y)):
        
        IPython.display.clear_output()
        print(f"Model {i+1}/{kfolds}")
        
        # Training model
        classifier.fit(X.iloc[train, ], y.iloc[train], )
        
        # Testing model
        prediction = classifier.predict(X.iloc[test,])
        
        # calculating and savings scores
        scores["f1"].append(                                   f1_score(y.iloc[test],prediction))
        scores["accuracy"].append(                       accuracy_score(y.iloc[test],prediction))
        scores["balanced_accuracy"].append(     balanced_accuracy_score(y.iloc[test],prediction))
        scores["precision"].append(                     precision_score(y.iloc[test],prediction))
        scores["average_precision"].append(     average_precision_score(y.iloc[test],prediction))
        scores["roc_auc"].append(                         roc_auc_score(y.iloc[test],prediction))
    
    IPython.display.clear_output()
    print(f"Crossvalidation on {kfolds} folds done in {round((time()-start_time),2)}s")
        
    return scores

In [ ]:
def get_mean_scores(scores_dict):
    means={}
    for score_name in scores_dict:
        means[score_name] = np.mean(scores_dict[score_name])
    return means

In [ ]:
def print_mean_scores(mean_scores_dict,precision=4):
    for score_name in mean_scores_dict:
        print(f"Mean {score_name} score is {round(mean_scores_dict[score_name]*100,precision)}%")

### Wyniki

Poniżej zamieszczam wyniki predykcji pokazanych wcześniej modeli. Dla kontrastu nauczyłem zmodyfikowane wersję klasyfikatorów jak i również te domyślne. Ze smutkiem muszę stwierdzić, że nie jestem najlepszy w strzelaniu, ponieważ parametry, które dobrałem znacznie pogarszają skutecznść każdego z modeli. Niemniej jednak by to stwierdzić musiałem sie posłóżyć pewnymi miarami. Są to:
* F1 
* Accuracy
* Balanced Accuracy
* Precision
* Average Precision
* ROC AUC

Wszystkie modele zostały poddane 10 krotnej kroswalidacji, więc przedstawione wyniki są średnią. Kroswalidacja pozwala dokładniej ocenić skutecznosć modelu oraz wyciągajac z nich takie informacje jak odchylenie standardowe wyników, co daje nam możliowść dyskusji na temat działania modelu w skrajnych przypadkach. 

### Random Forest

### Kroswalidacja modeli

In [ ]:
scores_rf_default = cv_classifier(rf_default)

In [ ]:
scores_rf_modified = cv_classifier(rf_modified)

In [ ]:
mean_scores_rf_default = get_mean_scores(scores_rf_default)
mean_scores_rf_modified = get_mean_scores(scores_rf_modified)

**Random forest default**

In [ ]:
print_mean_scores(mean_scores_rf_default,precision=2)

**Random forest modified**

In [ ]:
print_mean_scores(mean_scores_rf_modified,precision=2)

## Extreme Gradient Boosting

### Kroswalidacja modeli

In [ ]:
scores_xgb_default = cv_classifier(xgb_default)

In [ ]:
scores_xgb_modified = cv_classifier(xgb_modified)

In [ ]:
mean_scores_xgb_default = get_mean_scores(scores_xgb_default)
mean_scores_xgb_modified = get_mean_scores(scores_xgb_modified)

**XGBoost default**

In [ ]:
print_mean_scores(mean_scores_xgb_default,precision=2)

**XGBoost modified**

In [ ]:
print_mean_scores(mean_scores_xgb_modified,precision=2)

## Support Vector Machines

### Kroswalidacja modeli

**warning this takes a while**

In [ ]:
scores_svc_default = cv_classifier(svc_default)

In [ ]:
scores_svc_modified = cv_classifier(svc_modified)

In [ ]:
mean_scores_svc_default = get_mean_scores(scores_svc_default)
mean_scores_svc_modified = get_mean_scores(scores_svc_modified)

**SVM default**

In [ ]:
print_mean_scores(mean_scores_svc_default,precision=2)

**SVM modified**

In [ ]:
print_mean_scores(mean_scores_svc_modified,precision=2)

## Podsumowanie

Wyniki random forest oraz xgboost były dośyć zbliżone i szczerze mówiąc dosyć słabe. Jeszcze gorzej wypadł SVM, co pewnie wielu nie zdziwi. Ma okropnie długi czas uczenia, ponad minuta na model. Wypada dużo gorzej niż pozostałe algorytmy, gdzie 10 modeli xgboost zostało wyszkolone w 44s. Natomiast wyniki random forest oraz xgboost są dosyć zbliżone. Gdybym jednak miał wybrać jeden z tych trzech modeli, by dalej go dostrajać na pewno zdecydowałbym się na xgboosta. Między innymi dlatego, że czas uczenia i testowania byłby dużo krótszy niż w przypadku random forest, oraz prawdopodobnie z odpowiednimi parametrami xgboost będzie sobie radził lepiej niż random forest. 

# Część bonusowa - Regresja

### Przygotowanie danych

In [ ]:
data2 = pd.read_csv('allegro-api-transactions.csv')
data2 = data2.drop(['lp','date'], axis = 1)
data2.head()

Dane są prawie gotowe do procesu czenia, trzeba jedynie poprawić `it_location` w którym mogą pojawić się powtórki w stylu *Warszawa* i *warszawa*, a następnie zakodować zmienne kategoryczne

In [ ]:
data2.it_location = data2.it_location.str.lower()
data2.head()

In [ ]:
encoding_columns = ['categories','seller','it_location','main_category']

## Kodowanie zmiennych kategorycznych 

In [ ]:
import category_encoders
from sklearn.preprocessing import OneHotEncoder

### Podział danych 
Nie wykonam standardowego podziału na dane test i train, ponieważ w dalszej części dokumentu do oceny skutecznosci użytych kodowań posłużę się kroswalidacją. Pragnę zaznaczyć, ze prawdopodbnie najlepszą metodą tutaj byłoby rozbicie kategorii `categories` na 26 kolumn, zero-jedynkowych, jednak znacznie by to powiększyło rozmiar danych. Z dokładnie tego samego powodu nie wykonam one hot encodingu, tylko posłużę się kodowaniami, które nie powiększą rozmiatu danych.

In [ ]:
X = data2.drop('price', axis = 1)
y = data2.price

## Target encoding

In [ ]:
te = category_encoders.target_encoder.TargetEncoder(data2, cols = encoding_columns)
target_encoded = te.fit_transform(X,y)
target_encoded

## James-Stein Encoding

In [ ]:
js = category_encoders.james_stein.JamesSteinEncoder(cols = encoding_columns)
encoded_js = js.fit_transform(X,y)
encoded_js

## Cat Boost Encoding

In [ ]:
cb = category_encoders.cat_boost.CatBoostEncoder(cols = encoding_columns)
encoded_cb  = cb.fit_transform(X,y)
encoded_cb

## Testowanie

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import linear_model

In [ ]:
def cv_encoding(model,kfolds = 10, X = data.drop("RainTomorrow", axis = 1), y = data.RainTomorrow):
    start_time = time()
    
    scores ={}
    scores["r2_score"] = []
    scores['RMSE'] = []
    
    # Standard k-fold
    cv = KFold(n_splits=kfolds,shuffle=False,random_state=0)
    
    for i, (train, test) in enumerate(cv.split(X, y)):
        
        IPython.display.clear_output()
        print(f"Model {i+1}/{kfolds}")
        
        # Training model
        model.fit(X.iloc[train, ], y.iloc[train], )
        
        # Testing model
        prediction = model.predict(X.iloc[test,])
        
        # calculating and savings score
        scores['r2_score'].append(            r2_score(y.iloc[test],prediction))
        scores['RMSE'].append(      mean_squared_error(y.iloc[test],prediction))
        
    
    IPython.display.clear_output()
    print(f"Crossvalidation on {kfolds} folds done in {round((time()-start_time),2)}s")
        
    return scores

## Mierzenie skutecznosci kodowań
Zdecydowałem isę skorzystać z modelu regresji liniowej `Lasso` początkowo chciałem skorzystać z `Elastic Net`, ale jak się okazało zmienne nie sa ze sobą zbytnio powiązane, a to miał być główny powód do jego użycia.

In [ ]:
corr=data2.corr()
fig, ax=plt.subplots(figsize=(9,6))  
ax=sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, cmap="PiYG", center=0, vmin=-1, vmax=1)
ax.set_title('Korelacje zmiennych')
plt.show();

### Wybór modelu liniowego
Określam go w tym miejscu, ponieważ w dalszych częściach dokumentu będę z niego wielokrotnie korzystał przy kroswalidacji

In [ ]:
lasso = linear_model.Lasso()

## Wyniki target encodingu

In [ ]:
target_encoding_scores = cv_encoding(model = en,kfolds=20, X = target_encoded, y = y)

In [ ]:
target_encoding_scores_mean = get_mean_scores(target_encoding_scores)
target_encoding_scores_mean

## Wyniki James-Stein Encodingu

In [ ]:
js_encoding_scores = cv_encoding(en, 20, encoded_js, y)

In [ ]:
js_encoding_scores_mean = get_mean_scores(js_encoding_scores)
js_encoding_scores_mean

## Wyniki Cat Boost Encodingu

In [ ]:
cb_encoding_scores = cv_encoding(en, 20 ,encoded_cb, y)

In [ ]:
cb_encoding_scores_mean = get_mean_scores(cb_encoding_scores)
cb_encoding_scores_mean

## Porównanie

## Wyniki metryki r2

In [ ]:
r2_data = [target_encoding_scores["r2_score"], js_encoding_scores["r2_score"], cb_encoding_scores["r2_score"]]
labels = ["Target", " James-Stein", "Cat Boost"]
fig, ax = plt.subplots(figsize = (12,9))
ax.set_title('Wyniki  r2')
ax.boxplot(r2_data, labels = labels)
plt.show()

**Komentarz** 

Widać, że użycie kodowania Jamesa-Steina pozwoliło modelowi dużo lepiej się dopasować do danych, jednak stwarza to ewentaulny problem nadmiernego dopasowania się do danych. Warto by było sprawdzić czy przez ten sposób kodowania nie dochodzi do dużo silniejszego overfittingu. 

## Wynikii metryki RMSE

In [ ]:
rmse_data = [target_encoding_scores["RMSE"], js_encoding_scores["RMSE"], cb_encoding_scores["RMSE"]]
labels = ["Target", " James-Stein", "Cat Boost"]
fig, ax = plt.subplots(figsize = (12,9))
ax.set_title('Wyniki  RMSE w skali logarytmicznej')
ax.set_yscale('log')
ax.boxplot(rmse_data, labels = labels)
plt.show()

**Komentarz**

Najlepiej poradził sobie James-Stein Encoding, nic dziwnego ponieważ r2 wskazał nam już lepsze dopasowanie modelu.  

## Podsumowanie

Kodowanie Jamesa-Steina osiąga dużo lepsze wyniki niż pozostałe dwa przez mnie wybrane. O ile nie dochodzi w tym przypadku do overfittingu, to w tej grupie z pewnością wybrałbym właśnie to kodowanie. Warto się jednak zastanowić nad kodowaniem one-hot, które w tym przypadku wydaje się bardzo naturale, jednakże wiąże się z kilkukrotnym powiększeniem danych.